In [1]:
import string
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.image import ImageWriter
from pdfminer.layout import LAParams

import io
import os

import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import numpy as np
import pandas as pd

import json
from pathlib import *

import time

from random import randint

from nltk.corpus import wordnet

import operator

from nltk.chunk import tree2conlltags
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

import re

import numpy as np

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Pooja Ajit\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to C:\Users\Pooja
[nltk_data]     Ajit\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to C:\Users\Pooja
[nltk_data]     Ajit\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [48]:
def pdfRender():
    global documentSet
    global mydoc
    mydoc ={}
    pdf_files =[]
    allLines =[]
    FILE_PATH = Path(r'SLR3_EVALUATION_PAPERS/test/selected/QAset')
    #FILE_PATH = Path('E:/MasterThesis/FinalPapers')
    pdf_files = list(FILE_PATH.glob('*.pdf'))
    #An Array which stores the full text of each document
    documentSet = pdfparser(pdf_files)
    mydoc = dict(zip(pdf_files,documentSet))
    #print(len(documentSet))
    return documentSet

In [49]:
def processData(rawContents):    
    cleaned = tokenizeContent(rawContents)    
    cleaned1 = removeStopWordsFromTokenized(cleaned)    
    cleaned2 = performPorterStemmingOnContents(cleaned1)    
    cleaned3 = removePunctuationFromTokenized(cleaned2)    
    cleaned4 = convertItemsToLower(cleaned3)    
    return cleaned4    
        
def tokenizeContent(contentsRaw):    
    tokenized = nltk.tokenize.sent_tokenize(contentsRaw)    
    return tokenized    
    
def removeStopWordsFromTokenized(contentsTokenized):    
    stop_word_set = set(nltk.corpus.stopwords.words("english"))    
    filteredContents = [word for word in contentsTokenized if word not in stop_word_set]    
    return filteredContents    
    
def performPorterStemmingOnContents(contentsTokenized):    
    porterStemmer = nltk.stem.PorterStemmer()    
    filteredContents = [porterStemmer.stem(word) for word in contentsTokenized]    
    return filteredContents    
    
def removePunctuationFromTokenized(contentsTokenized):    
    excludePuncuation = set(string.punctuation)    
    
    # manually add additional punctuation to remove    
    doubleSingleQuote = '\'\''    
    doubleDash = '--'    
    doubleTick = '``'    
    
    excludePuncuation.add(doubleSingleQuote)    
    excludePuncuation.add(doubleDash)    
    excludePuncuation.add(doubleTick)    
    
    filteredContents = [word for word in contentsTokenized if word not in excludePuncuation]    
    return filteredContents    
    
def convertItemsToLower(contentsRaw):    
    filteredContents = [term.lower() for term in contentsRaw]    
    return filteredContents


In [50]:
global fileName
def pdfparser(pdffileS):
    global finalDocumentSet
    finalDocumentSet = []
    global pdfEx
    pdfEx = []
    global files
    global fullText
    files = []
    fileName = []
    for pdffile in pdffileS:
        #full= fullText
        # Create a example words list(Please add all the related keywords needed)
        words_list =[ "Abstract", "Introduction", "INTRODUCTION", "Motivation", "Description", "Overview", "Background", "BACKGROUND", "METHODOLOGY", "Methods", 
                      "FUTURE DIRECTIONS", "Conclusion", "Conclusions", "CONCLUSION",  "CONCLUSIONS", "Concluding", "CONCLUDING", "Summary and conclusions", "Summary", 
                      "Acknowledgements", "Discussions and conclusions", "References", "REFERENCES"]
        #print(words_list)
        pdfName = os.path.basename(pdffile)
        files.append(pdfName)
        fileName.append(pdfName)
        with open(pdffile, mode='rb') as f:
            fullText = np.array([])
            print(pdfName)
            #files.append(pdfName)
            #documents = fullText
            #words_list = []
            #print(words_list)
            #fp = open(data, 'rb')
            rsrcmgr = PDFResourceManager()
            retstr = io.StringIO()
            codec = 'utf-8'
            laparams = LAParams()
            data =[]
            details_page = []
            abstract = []
            device = TextConverter(rsrcmgr, retstr, laparams=laparams)
            # Create a PDF interpreter object.
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            # Process each page contained in the document.
            count = 0
            for page in PDFPage.get_pages(f):
                interpreter.process_page(page)
                data = retstr.getvalue()
                details_page.append(data)

            #print("There are", len(words_list), "in the words list")
            stri = " "
            details = stri.join(details_page)
            words = details.split()
            place = []
            dummy_check = []
            removed_words = []

            print(words_list)
            for c, a in enumerate(words):
                for b in words_list:
                    if b == a and b not in dummy_check:
                        print(b, a)
                        place.append(details.find("{}".format(b)))
                        dummy_check.append(b)
                    #  place.append(words.index(a))
                    elif b not in words:
                        print(b)
                        removed_words.append(b)
                        words_list.remove(b)
                        print("The word", b, "was not found in the pdf file")

            #print(list(zip(words_list, place)))
            final_array = list(zip(words_list, place))
            #final_array.sort()
            final_array.sort(key=operator.itemgetter(1))
            # print("Sorting the final array")
            #print(final_array)

            # print("Extracting the relevant texts from pdf")
            # print(" ")
            print(final_array)
            if len(final_array) > 1:
                listint = final_array[0]
                list2int = final_array[1]
                counter = 0

                for each in (final_array):
                    if counter < len(final_array) - 2:
                        new = (details.split(listint[0])[1].split(list2int[0])[0])
                        #new = sent_tokenize(new)
                        #print(listint[0], ":", [' '.join(new)])
                        #print(" ")
                        #print(new)
                        #documents.append(new)
                        fullText = np.append(fullText, new)
                        counter = counter + 1
                        listint = final_array[0 + counter]
                        list2int = final_array[1 + counter]

                    elif counter < len(final_array) - 1:
                        new = (details.split(final_array[counter][0])[1].split(final_array[counter + 1][0])[0])
                        #new = sent_tokenize(new)
                        #documents.append(new)
                        fullText = np.append(fullText, new)
                        #print(final_array[counter][0], ":", [' '.join(new)])
                        #print(" ")
                        counter = counter + 1

                    else:
                        new = (details.split(final_array[counter][0])[1])
                        #new = sent_tokenize(new)
                        #documents.append(new)
                        fullText = np.append(fullText, new)
                        #print(final_array[counter][0], ":", [' '.join(new)])
                        #print(" ")
            else:
                new = (details.split(final_array[0][0])[1])
                # new = sent_tokenize(new)
                #documents.append(new)
                fullText = np.append(fullText, new)
                # print(final_array[counter][0], ":", [' '.join(new)])
                # print(" ")
                
        #finalDocumentSet = {pdfName : fullText}
        
        data=finalDocumentSet.append(fullText)
        myName=pdfEx.append(pdfName)
        #print("Testing==",finalDocumentSet)
        #data = finalDocumentSet.get(pdfName)
        #finalDocumentSet = finalDocumentSet
        data = str(data)
        
        data = processData(data)
        #data = data.replace(r'\\n', "")
        data = [i.replace('\\n', "") for i in data]
        data = [i.replace('\\x0', "") for i in data]
        words_list = words_list + removed_words
        print("Updated words list:")
        print(words_list)

    #print(len(finalDocumentSet))
    
    #mydoc = dict(zip(myName,data))
    return data


In [51]:
pdfRender()

A DMAIRC approach to lead time reduction in an aerospace engine assembly process.pdf
['Abstract', 'Introduction', 'INTRODUCTION', 'Motivation', 'Description', 'Overview', 'Background', 'BACKGROUND', 'METHODOLOGY', 'Methods', 'FUTURE DIRECTIONS', 'Conclusion', 'Conclusions', 'CONCLUSION', 'CONCLUSIONS', 'Concluding', 'CONCLUDING', 'Summary and conclusions', 'Summary', 'Acknowledgements', 'Discussions and conclusions', 'References', 'REFERENCES']
INTRODUCTION
The word INTRODUCTION was not found in the pdf file
Description
The word Description was not found in the pdf file
Background
The word Background was not found in the pdf file
METHODOLOGY
The word METHODOLOGY was not found in the pdf file
FUTURE DIRECTIONS
The word FUTURE DIRECTIONS was not found in the pdf file
CONCLUSION
The word CONCLUSION was not found in the pdf file
Concluding
The word Concluding was not found in the pdf file
Summary and conclusions
The word Summary and conclusions was not found in the pdf file
Acknowledgement

Introduction Introduction
Conclusions Conclusions
References References
Motivation Motivation
Methods Methods
[('Introduction', 2140), ('Motivation', 259340), ('Methods', 388027), ('Conclusions', 455603), ('References', 461339)]
Updated words list:
['Introduction', 'Motivation', 'Methods', 'Conclusions', 'References', 'Abstract', 'INTRODUCTION', 'Description', 'Background', 'METHODOLOGY', 'FUTURE DIRECTIONS', 'CONCLUSION', 'Concluding', 'Summary and conclusions', 'Acknowledgements', 'REFERENCES', 'Overview', 'Conclusion', 'CONCLUSIONS', 'Summary', 'BACKGROUND', 'CONCLUDING', 'Discussions and conclusions']
Adopting agile practices when developing software for use in the medical domain.pdf
['Abstract', 'Introduction', 'INTRODUCTION', 'Motivation', 'Description', 'Overview', 'Background', 'BACKGROUND', 'METHODOLOGY', 'Methods', 'FUTURE DIRECTIONS', 'Conclusion', 'Conclusions', 'CONCLUSION', 'CONCLUSIONS', 'Concluding', 'CONCLUDING', 'Summary and conclusions', 'Summary', 'Acknowledgements'

Introduction Introduction
Description Description
Summary Summary
References References
Methods Methods
[('Introduction', 2281), ('Description', 53845), ('Methods', 159612), ('Summary', 393996), ('References', 481557)]
Updated words list:
['Introduction', 'Description', 'Methods', 'Summary', 'References', 'Abstract', 'INTRODUCTION', 'Overview', 'BACKGROUND', 'FUTURE DIRECTIONS', 'Conclusions', 'CONCLUSIONS', 'CONCLUDING', 'Acknowledgements', 'REFERENCES', 'Motivation', 'Background', 'Conclusion', 'Concluding', 'Discussions and conclusions', 'METHODOLOGY', 'CONCLUSION', 'Summary and conclusions']
AnEmpiricalInvestigationoftheChallengesoftheExistingtoolsusedinGSD.pdf
['Abstract', 'Introduction', 'INTRODUCTION', 'Motivation', 'Description', 'Overview', 'Background', 'BACKGROUND', 'METHODOLOGY', 'Methods', 'FUTURE DIRECTIONS', 'Conclusion', 'Conclusions', 'CONCLUSION', 'CONCLUSIONS', 'Concluding', 'CONCLUDING', 'Summary and conclusions', 'Summary', 'Acknowledgements', 'Discussions and conc

Abstract Abstract
Introduction Introduction
Conclusion Conclusion
Conclusions Conclusions
Acknowledgements Acknowledgements
References References
Overview Overview
[('Abstract', 2376), ('Introduction', 4208), ('Overview', 179055), ('Conclusion', 511804), ('Conclusions', 587963), ('Acknowledgements', 749014), ('References', 836525)]
Updated words list:
['Abstract', 'Introduction', 'Overview', 'Conclusion', 'Conclusions', 'Acknowledgements', 'References', 'INTRODUCTION', 'Description', 'Background', 'METHODOLOGY', 'FUTURE DIRECTIONS', 'CONCLUSION', 'Concluding', 'Summary and conclusions', 'Discussions and conclusions', 'REFERENCES', 'Motivation', 'BACKGROUND', 'CONCLUSIONS', 'Summary', 'Methods', 'CONCLUDING']
AsystematicLiteratureReviewSoftwarerequirementsprioritizationtechniques.pdf
['Abstract', 'Introduction', 'INTRODUCTION', 'Motivation', 'Description', 'Overview', 'Background', 'BACKGROUND', 'METHODOLOGY', 'Methods', 'FUTURE DIRECTIONS', 'Conclusion', 'Conclusions', 'CONCLUSION', 'C

References References
[('References', 28871)]
Updated words list:
['References', 'Abstract', 'INTRODUCTION', 'Description', 'Background', 'METHODOLOGY', 'FUTURE DIRECTIONS', 'Conclusions', 'CONCLUSIONS', 'CONCLUDING', 'Summary', 'Discussions and conclusions', 'REFERENCES', 'Introduction', 'Overview', 'Methods', 'CONCLUSION', 'Summary and conclusions', 'Motivation', 'Conclusion', 'Acknowledgements', 'BACKGROUND', 'Concluding']
Elder2015_Article_ASystematicAssessmentOfTheUseO.pdf
['Abstract', 'Introduction', 'INTRODUCTION', 'Motivation', 'Description', 'Overview', 'Background', 'BACKGROUND', 'METHODOLOGY', 'Methods', 'FUTURE DIRECTIONS', 'Conclusion', 'Conclusions', 'CONCLUSION', 'CONCLUSIONS', 'Concluding', 'CONCLUDING', 'Summary and conclusions', 'Summary', 'Acknowledgements', 'Discussions and conclusions', 'References', 'REFERENCES']
Abstract
The word Abstract was not found in the pdf file
Motivation
The word Motivation was not found in the pdf file
Overview
The word Overview was not 

...

Introduction Introduction
Conclusions Conclusions
References References
[('Introduction', 2408), ('Conclusions', 277674), ('References', 471084)]
Updated words list:
['Introduction', 'Conclusions', 'References', 'Abstract', 'INTRODUCTION', 'Description', 'Background', 'METHODOLOGY', 'FUTURE DIRECTIONS', 'CONCLUSION', 'Concluding', 'Summary and conclusions', 'Acknowledgements', 'REFERENCES', 'Motivation', 'BACKGROUND', 'Conclusion', 'CONCLUSIONS', 'Summary', 'Overview', 'CONCLUDING', 'Methods', 'Discussions and conclusions']
Pairwise testing for software product lines Comparison of two approaches.pdf
['Abstract', 'Introduction', 'INTRODUCTION', 'Motivation', 'Description', 'Overview', 'Background', 'BACKGROUND', 'METHODOLOGY', 'Methods', 'FUTURE DIRECTIONS', 'Conclusion', 'Conclusions', 'CONCLUSION', 'CONCLUSIONS', 'Concluding', 'CONCLUDING', 'Summary and conclusions', 'Summary', 'Acknowledgements', 'Discussions and conclusions', 'References', 'REFERENCES']
INTRODUCTION
The word INTRODU

Introduction Introduction
Conclusion Conclusion
References References
[('Introduction', 733), ('Methods', 3155), ('Conclusion', 146115), ('References', 147167)]
Updated words list:
['Introduction', 'Methods', 'Conclusion', 'References', 'Abstract', 'INTRODUCTION', 'Description', 'Background', 'METHODOLOGY', 'FUTURE DIRECTIONS', 'Conclusions', 'CONCLUSIONS', 'CONCLUDING', 'Summary', 'Discussions and conclusions', 'REFERENCES', 'Motivation', 'BACKGROUND', 'CONCLUSION', 'Summary and conclusions', 'Overview', 'Concluding', 'Acknowledgements']
Software Development Processes for Games A Systematic Literature Review.pdf
['Abstract', 'Introduction', 'INTRODUCTION', 'Motivation', 'Description', 'Overview', 'Background', 'BACKGROUND', 'METHODOLOGY', 'Methods', 'FUTURE DIRECTIONS', 'Conclusion', 'Conclusions', 'CONCLUSION', 'CONCLUSIONS', 'Concluding', 'CONCLUDING', 'Summary and conclusions', 'Summary', 'Acknowledgements', 'Discussions and conclusions', 'References', 'REFERENCES']
Abstract
The wo

Abstract Abstract
Introduction Introduction
Conclusions Conclusions
References References
[('Abstract', 2267), ('Introduction', 7438), ('Conclusions', 388769), ('References', 783808)]
Updated words list:
['Abstract', 'Introduction', 'Conclusions', 'References', 'INTRODUCTION', 'Description', 'Background', 'METHODOLOGY', 'FUTURE DIRECTIONS', 'CONCLUSION', 'Concluding', 'Summary and conclusions', 'Acknowledgements', 'REFERENCES', 'Motivation', 'BACKGROUND', 'Conclusion', 'CONCLUSIONS', 'Summary', 'Overview', 'CONCLUDING', 'Methods', 'Discussions and conclusions']
Study of factors influencing the adoption of agile processes when using Wikis.pdf
['Abstract', 'Introduction', 'INTRODUCTION', 'Motivation', 'Description', 'Overview', 'Background', 'BACKGROUND', 'METHODOLOGY', 'Methods', 'FUTURE DIRECTIONS', 'Conclusion', 'Conclusions', 'CONCLUSION', 'CONCLUSIONS', 'Concluding', 'CONCLUDING', 'Summary and conclusions', 'Summary', 'Acknowledgements', 'Discussions and conclusions', 'References', 

INTRODUCTION INTRODUCTION
BACKGROUND BACKGROUND
Conclusion Conclusion
Summary Summary
CONCLUSIONS CONCLUSIONS
REFERENCES REFERENCES
[('INTRODUCTION', 4244), ('BACKGROUND', 8046), ('Conclusion', 55753), ('CONCLUSIONS', 80674), ('Summary', 431610), ('REFERENCES', 499009)]
Updated words list:
['INTRODUCTION', 'BACKGROUND', 'Conclusion', 'CONCLUSIONS', 'Summary', 'REFERENCES', 'Abstract', 'Motivation', 'Overview', 'METHODOLOGY', 'FUTURE DIRECTIONS', 'Conclusions', 'Concluding', 'Summary and conclusions', 'Acknowledgements', 'References', 'Introduction', 'Description', 'Methods', 'CONCLUSION', 'CONCLUDING', 'Discussions and conclusions', 'Background']
Tailoring software development methodologies in practice A case study.pdf
['Abstract', 'Introduction', 'INTRODUCTION', 'Motivation', 'Description', 'Overview', 'Background', 'BACKGROUND', 'METHODOLOGY', 'Methods', 'FUTURE DIRECTIONS', 'Conclusion', 'Conclusions', 'CONCLUSION', 'CONCLUSIONS', 'Concluding', 'CONCLUDING', 'Summary and conclusions

['none']

In [52]:
len(finalDocumentSet)

47

In [53]:
len(files)


47

In [54]:
files[0]

'A DMAIRC approach to lead time reduction in an aerospace engine assembly process.pdf'

In [55]:
#create a dictionary to save the pdf name and the objectives for QA questions
final_dataset_QA = {'fileName': [], 'content': [] }
for a,b in zip(files, finalDocumentSet):    
    final_dataset_QA['fileName'].append(a)
    final_dataset_QA['content'].append(b)

In [56]:
#export the results to .xslx
from pandas import DataFrame

df = DataFrame(final_dataset_QA, columns= ['fileName', 'content'])
export_excel = df.to_excel (r'E:\MasterThesis\EVALUATION\SLR_evaluation\New folder\SLR3\QA_evaluation\SLR3_Dataset.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path



In [57]:
#extract introduction, methods, conclusion portions of the paper
extractedContent = []
global keyword
def extractIntroduction():
    count = 0
    for i in range(len(finalDocumentSet)):
        doc = str(finalDocumentSet[i])
        intro=re.search("|".join([r'Background(.*?)Methods', r'2. Study objective(.*?)3. Methods', r'Introduction(.*?)Methods', r'Purpose(.*?)Methods', r'Introduction(.*?)Overview', r'Introduction(.*?)Method', r'Background(.*?)Main Text', r'Introduction(.*?)Methodology', r'Introduction(.*?) Related work', r'Context(.*?)Methods', r'Introduction(.*?)Methodology', r'Objectives(.*?)Methods', r'Introduction(.*?)Study Objective', r'Objective(.*?)Method', r'Introduction(.*?)Background' ]), str(doc)).group()           
        print("Introduction is extracted successfully")
        extractedContent.append(intro) 
        return extractedContent 

In [58]:
other_content = []
for i in range(len(finalDocumentSet)):
    content = str(finalDocumentSet[i])
    #Text pre-processing
    content = content.lower()
    content = re.sub(r'\d+', '', content)
    #data = re.sub(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+','', data)
    content = content.replace("\n", "")
    #data = data.strip()
    content = sent_tokenize(content)
    other_content.append(content)

In [59]:
len(other_content)

47

In [94]:
other_content[0]

["['\\npurpose – problem solving and continuous process improvement are key elements to achieve\\nbusiness excellence.",
 'many problem solving and process improvement methodologies have been\\nproposed and adopted by organisations, with dmaic being the most widely used.',
 'the purpose of this\\npaper is to present an empirical application of a modiﬁed version of dmaic which enabled a\\nworld-class organisation to achieve an optimum reduction in the lead time of its aerospace engine\\nassembly process.\\ndesign/methodology/approach – the paper reviews the most commonly used problem solving\\nand process improvement methodologies and speciﬁcally, dmaic, its variations and limitations.\\nbased on this, it presents deﬁne, measure, analyse, improve, review, control (dmairc).',
 'finally,\\ndmairc is empirically applied through a case study, in a world-class manufacturing organisation.\\nfindings – the results obtained from the case study indicate that dmairc is an effective alternative\\n

In [60]:
pip install rank-bm25

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [61]:
from rank_bm25 import BM25Okapi

In [62]:
def create_tokens_questions(question):
    tokenized_question = question.split()
    create_chunks = nltk.ne_chunk(nltk.pos_tag(tokenized_question))
    assign_pos_tags = tree2conlltags(create_chunks)
    return assign_pos_tags

In [96]:
def extract_pos(question):
    noun_add = []
    assign_pos_tags = create_tokens_questions(question)
    print(assign_pos_tags)
    for i in range(0, len(assign_pos_tags) - 1):
        if (assign_pos_tags[i][1] == 'NN' and assign_pos_tags[i + 1][1] == 'NN'):
            extract_nouns = assign_pos_tags[i] + assign_pos_tags[i + 1]
            noun_add.append(extract_nouns)
        elif assign_pos_tags[i][1] == 'NNS':
            extract_nouns = assign_pos_tags[i]
            print("The word", extract_nouns[0], "is extracted and added to the query list")
            noun_add.append(extract_nouns[0])
        elif assign_pos_tags[i][1] == 'NN':
            extract_nouns = assign_pos_tags[i]
            print("The word", extract_nouns[0], "is extracted and added to the query list")
            noun_add.append(extract_nouns[0])
    return noun_add

In [97]:
introFinalData = []
global keyword
def extractIntroduction(inputDoc):
    for j in range(len(inputDoc)):
        question = "what is the aim and objective of this paper?"
        tokenized_corpus = [doc.split(" ") for doc in inputDoc[j]]
        bm25 = BM25Okapi(tokenized_corpus)
        noun_add = extract_pos(question)
        #print(noun_add)
        get_final_results = bm25.get_top_n(noun_add, inputDoc[j], n=5)
        introFinalData.append(get_final_results)
        #print(get_final_results)
    return introFinalData


In [98]:
otherContent = []
global keyword
def extractMethods(inputDoc):
    for j in range(len(inputDoc)):
        question = "what are the methods or method or methodology used in this paper?"
        tokenized_corpus = [doc.split(" ") for doc in inputDoc[j]]
        bm25 = BM25Okapi(tokenized_corpus)
        noun_add = extract_pos(question)
        #print(noun_add)
        get_final_results = bm25.get_top_n(noun_add, inputDoc[j], n=10)
        otherContent.append(get_final_results)
        #print(get_final_results)
    return otherContent

In [99]:
Conclu = []
global keyword
def extractConclusion(inputDoc):
    for j in range(len(inputDoc)):
        question = "what is the conclusions and discussions and justifications of this paper?"
        tokenized_corpus = [doc.split(" ") for doc in inputDoc[j]]
        bm25 = BM25Okapi(tokenized_corpus)
        noun_add = extract_pos(question)
        #print(noun_add)
        get_final_results = bm25.get_top_n(noun_add, inputDoc[j], n=10)
        Conclu.append(get_final_results)
        #print(get_final_results)
    return Conclu

In [100]:
#To fetch the conclusion for each paper
QA1 = extractIntroduction(other_content)

[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('and', 'CC', 'O'), ('objective', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word aim is extracted and added to the query list
The word objective is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('and', 'CC', 'O'), ('objective', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word aim is extracted and added to the query list
The word objective is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('and', 'CC', 'O'), ('objective', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word aim is extracted and added to the query list
The word objective is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('a

[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('and', 'CC', 'O'), ('objective', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word aim is extracted and added to the query list
The word objective is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('and', 'CC', 'O'), ('objective', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word aim is extracted and added to the query list
The word objective is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('and', 'CC', 'O'), ('objective', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word aim is extracted and added to the query list
The word objective is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('a

In [101]:
len(QA1)

47

In [102]:
QA1

[['the ﬁnal results achieved through this project\\nindicate that the aim was not only achieved but exceeded through a reduction of about\\n percent.',
  'the ﬁnal results achieved through this project\\nindicate that the aim was not only achieved but exceeded through a reduction of about\\n percent.',
  'table i summarises the historical measures\\nand target set for this improvement project as well as the lead time and variability\\nachieved through the different trials performed.\\n\\nin general terms, the aim of this improvement project was to reduce the engine\\nassembly lead time by  percent.',
  'table i summarises the historical measures\\nand target set for this improvement project as well as the lead time and variability\\nachieved through the different trials performed.\\n\\nin general terms, the aim of this improvement project was to reduce the engine\\nassembly lead time by  percent.',
  'as previously commented, the aim of this improvement project was to\\n\\nfigure .\\nt

In [103]:
#To fetch the methods for each paper
QA2 = extractMethods(other_content)

[('what', 'WDT', 'O'), ('are', 'VBP', 'O'), ('the', 'DT', 'O'), ('methods', 'NNS', 'O'), ('or', 'CC', 'O'), ('method', 'NN', 'O'), ('or', 'CC', 'O'), ('methodology', 'NN', 'O'), ('used', 'VBN', 'O'), ('in', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word methods is extracted and added to the query list
The word method is extracted and added to the query list
The word methodology is extracted and added to the query list
[('what', 'WDT', 'O'), ('are', 'VBP', 'O'), ('the', 'DT', 'O'), ('methods', 'NNS', 'O'), ('or', 'CC', 'O'), ('method', 'NN', 'O'), ('or', 'CC', 'O'), ('methodology', 'NN', 'O'), ('used', 'VBN', 'O'), ('in', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word methods is extracted and added to the query list
The word method is extracted and added to the query list
The word methodology is extracted and added to the query list
[('what', 'WDT', 'O'), ('are', 'VBP', 'O'), ('the', 'DT', 'O'), ('methods', 'NNS', 'O'), ('or', 'CC', 'O'), ('method', '

[('what', 'WDT', 'O'), ('are', 'VBP', 'O'), ('the', 'DT', 'O'), ('methods', 'NNS', 'O'), ('or', 'CC', 'O'), ('method', 'NN', 'O'), ('or', 'CC', 'O'), ('methodology', 'NN', 'O'), ('used', 'VBN', 'O'), ('in', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word methods is extracted and added to the query list
The word method is extracted and added to the query list
The word methodology is extracted and added to the query list
[('what', 'WDT', 'O'), ('are', 'VBP', 'O'), ('the', 'DT', 'O'), ('methods', 'NNS', 'O'), ('or', 'CC', 'O'), ('method', 'NN', 'O'), ('or', 'CC', 'O'), ('methodology', 'NN', 'O'), ('used', 'VBN', 'O'), ('in', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word methods is extracted and added to the query list
The word method is extracted and added to the query list
The word methodology is extracted and added to the query list
[('what', 'WDT', 'O'), ('are', 'VBP', 'O'), ('the', 'DT', 'O'), ('methods', 'NNS', 'O'), ('or', 'CC', 'O'), ('method', '

[('what', 'WDT', 'O'), ('are', 'VBP', 'O'), ('the', 'DT', 'O'), ('methods', 'NNS', 'O'), ('or', 'CC', 'O'), ('method', 'NN', 'O'), ('or', 'CC', 'O'), ('methodology', 'NN', 'O'), ('used', 'VBN', 'O'), ('in', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word methods is extracted and added to the query list
The word method is extracted and added to the query list
The word methodology is extracted and added to the query list
[('what', 'WDT', 'O'), ('are', 'VBP', 'O'), ('the', 'DT', 'O'), ('methods', 'NNS', 'O'), ('or', 'CC', 'O'), ('method', 'NN', 'O'), ('or', 'CC', 'O'), ('methodology', 'NN', 'O'), ('used', 'VBN', 'O'), ('in', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word methods is extracted and added to the query list
The word method is extracted and added to the query list
The word methodology is extracted and added to the query list
[('what', 'WDT', 'O'), ('are', 'VBP', 'O'), ('the', 'DT', 'O'), ('methods', 'NNS', 'O'), ('or', 'CC', 'O'), ('method', '

In [104]:
QA2

[['() proposed the minidmaic methodology to\\nsimplify the traditional dmaic method in order to address causes and resolutions of\\nproblems, particularly, in software development projects.',
  '() proposed the minidmaic methodology to\\nsimplify the traditional dmaic method in order to address causes and resolutions of\\nproblems, particularly, in software development projects.',
  '() proposed the minidmaic methodology to\\nsimplify the traditional dmaic method in order to address causes and resolutions of\\nproblems, particularly, in software development projects.',
  '() suggest dmaic and design for six sigma (dfss) as the two\\nmost commonly used methods by organisations to implement six sigma.',
  '() suggest dmaic and design for six sigma (dfss) as the two\\nmost commonly used methods by organisations to implement six sigma.',
  '() suggest dmaic and design for six sigma (dfss) as the two\\nmost commonly used methods by organisations to implement six sigma.',
  'similarly as dma

In [105]:
QA2[1]

['sqfd originated in japan in  as a method to\\nimprove the quality of embedded software.',
 '(a) developed a method for setting target values of\\nengineering attributes by incorporating the technical trend and time.\\n\\nseveral empirical studies reported applications of qfd in software development.',
 '() proposed a hybrid algorithm that incorporates the least\\nsquares regression method into fuzzy linear regression in order to investigate functional\\nrelationships between customer requirements and technical attributes.',
 'as a front-\\nend technique, sqfd is an adaptation of the hoq, the most commonly used matrix in the\\ntraditional qfd methodology (haag et al.',
 'as a front-\\nend technique, sqfd is an adaptation of the hoq, the most commonly used matrix in the\\ntraditional qfd methodology (haag et al.',
 ').\\n\\nfuzzy methods are widely used in qfd to deal with the subjectivity and ambiguity of\\nevaluations regarding the customer requirements and technical attributes and t

In [106]:
#To fetch the conclusion for each paper
QA3 = extractConclusion(other_content)

[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusions', 'NNS', 'O'), ('and', 'CC', 'O'), ('discussions', 'NNS', 'O'), ('and', 'CC', 'O'), ('justifications', 'NNS', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word conclusions is extracted and added to the query list
The word discussions is extracted and added to the query list
The word justifications is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusions', 'NNS', 'O'), ('and', 'CC', 'O'), ('discussions', 'NNS', 'O'), ('and', 'CC', 'O'), ('justifications', 'NNS', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word conclusions is extracted and added to the query list
The word discussions is extracted and added to the query list
The word justifications is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusions', 'NNS', 'O'), ('and', 'CC', 'O'), (

[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusions', 'NNS', 'O'), ('and', 'CC', 'O'), ('discussions', 'NNS', 'O'), ('and', 'CC', 'O'), ('justifications', 'NNS', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word conclusions is extracted and added to the query list
The word discussions is extracted and added to the query list
The word justifications is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusions', 'NNS', 'O'), ('and', 'CC', 'O'), ('discussions', 'NNS', 'O'), ('and', 'CC', 'O'), ('justifications', 'NNS', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word conclusions is extracted and added to the query list
The word discussions is extracted and added to the query list
The word justifications is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusions', 'NNS', 'O'), ('and', 'CC', 'O'), (

[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusions', 'NNS', 'O'), ('and', 'CC', 'O'), ('discussions', 'NNS', 'O'), ('and', 'CC', 'O'), ('justifications', 'NNS', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word conclusions is extracted and added to the query list
The word discussions is extracted and added to the query list
The word justifications is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusions', 'NNS', 'O'), ('and', 'CC', 'O'), ('discussions', 'NNS', 'O'), ('and', 'CC', 'O'), ('justifications', 'NNS', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word conclusions is extracted and added to the query list
The word discussions is extracted and added to the query list
The word justifications is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusions', 'NNS', 'O'), ('and', 'CC', 'O'), (

In [107]:
len(QA1)

47

In [108]:
len(QA2)

47

In [109]:
len(QA3)

47

In [110]:
global data_cleaned
def clean(data):
    data_cleaned=[]
    for line in data:
        line = line.lower()
        line = re.sub(r'\d+', '', line)
        #line = re.sub(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+','', line)
        #line = line.translate(str.maketrans('','', string.punctuation))
        #line = line.strip()
        line = line.replace("\n", "")
        line = line.replace("\\n", "")
        #line = sent_tokenize(line)
        
        data_cleaned.append(line)
    return data_cleaned
    

In [111]:
QA1_Data = []
for k in range(len(QA1)):
    CleanedData = (QA1[k])
    CleanedData = clean(CleanedData)
    QA1_Data.append(CleanedData)


In [121]:
%store QA1_Data

Stored 'QA1_Data' (list)


In [122]:
%store QA2_Data

Stored 'QA2_Data' (list)


In [123]:
%store QA3_Data

Stored 'QA3_Data' (list)


In [115]:
QA2_Data = []
for l in range(len(QA2)):
    CleanedData = (QA2[l])
    CleanedData = clean(CleanedData)
    QA2_Data.append(CleanedData)

In [116]:
QA3_Data = []
for m in range(len(QA3)):
    CleanedData = (QA3[m])
    CleanedData = clean(CleanedData)
    QA3_Data.append(CleanedData)

In [117]:
#create a dictionary to save the pdf name and the respective answers for the QA1
#Is the aim and the objective stated clearly?
QA1_Ans = {'PDFName': [], 'QA1_ans': [] }
for c,d in zip(files, QA1_Data):    
    QA1_Ans['PDFName'].append(c)
    QA1_Ans['QA1_ans'].append(d)


In [118]:
#create a dictionary to save the pdf name and the respective answers for the QA2
#This can answer two questions partially 
#Adequate description of the sample used and the methods for identifying and recruiting the sample
#Adequate description of methods used to analyse the data (partially)
QA2_Ans = {'PDFName': [], 'QA2_ans': [] }
for e,f in zip(files, QA2_Data):    
    QA2_Ans['PDFName'].append(e)
    QA2_Ans['QA2_ans'].append(f)

In [119]:
#create a dictionary to save the pdf name and the respective answers for the QA3
#Does the study provide clearly stated findings with credible results and justified conclusions
QA3_Ans = {'PDFName': [], 'QA3_ans': [] }
for g,h in zip(files, QA3_Data):    
    QA3_Ans['PDFName'].append(g)
    QA3_Ans['QA3_ans'].append(h)


In [44]:
df3 = DataFrame(QA3_Ans, columns= ['PDFName', 'QA3_ans'])
export_excel = df3.to_excel (r'E:\MasterThesis\EVALUATION\SLR_evaluation\New folder\SLR3\QA_evaluation\QA3_Answer.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path


In [120]:
#export the answers to the QA1,QA2,QA3 to .xslx

#the final results are exported to the excel to let the user read it and score/assess the paper accordingly
from pandas import DataFrame

df1 = DataFrame(QA1_Ans, columns= ['PDFName', 'QA1_ans'])
export_excel = df1.to_excel (r'E:\MasterThesis\EVALUATION\SLR_evaluation\New folder\SLR3\QA_evaluation\QA1_Answer.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path

df2 = DataFrame(QA2_Ans, columns= ['PDFName', 'QA2_ans'])
export_excel = df2.to_excel (r'E:\MasterThesis\EVALUATION\SLR_evaluation\New folder\SLR3\QA_evaluation\QA2_Answer.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path

df3 = DataFrame(QA3_Ans, columns= ['PDFName', 'QA3_ans'])
export_excel = df3.to_excel (r'E:\MasterThesis\EVALUATION\SLR_evaluation\New folder\SLR3\QA_evaluation\QA3_Answer.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path
